In [1]:
# import dependencies
import pandas as pd
import numpy as np

In [2]:
# read data file and do the initial cleansing by replacing '.' or '-1' (masked data) with NaN
df_campus = pd.read_csv('CAMPSTAAR2.dat')
df_campus.replace('-1',np.nan,inplace=True)
df_campus.replace('.',np.nan,inplace=True)

In [3]:
# quick data exploration shows it is a big and wide table
df_campus.shape

(8757, 1513)

In [4]:
# the column headers are coded and not user friendly for further exploration
df_campus.head()

,CAMPUS,CA00AR01016D,CA00AR01S16N,CB00AR01016D,CB00AR01S16N,CW00AR01016D,CW00AR01S16N,CH00AR01016D,CH00AR01S16N,CI00AR01016D,...,CE00A003A17R,CS00A003017D,CS00A003A17N,CS00A003A17R,CR00A003017D,CR00A003A17N,CR00A003A17R,CL00A003017D,CL00A003A17N,CL00A003A17R
0,1902001,108,78,NaN,NaN,88,70,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
1,1902041,116,100,NaN,NaN,92,80,12,9,NaN,...,10,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
2,1902103,115,104,NaN,NaN,92,88,13,9,NaN,...,18,38,7,18,NaN,NaN,NaN,NaN,NaN,NaN
3,1903001,207,173,10,6,173,147,15,13,NaN,...,6,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
4,1903041,284,224,20,9,226,184,22,16,NaN,...,13,101,7,7,286,7,2,0,NaN,NaN


In [5]:
# read 3 indiviudal header mapping tables and combine them into one mapping table
df_header1 = pd.read_csv('header_mapping_1.csv')
df_header2 = pd.read_csv('header_mapping_2.csv')
df_header3 = pd.read_csv('header_mapping_3.csv')
df_header = pd.concat([df_header1,df_header2,df_header3],ignore_index=True)

In [6]:
# create a dictionary for the column header mapping table, and use it to rename the column headers
header_dict = {}
for i in range(len(df_header)):
    header_dict[df_header.loc[i,'NAME']] = df_header.loc[i,'LABEL']    
df_campus = df_campus.rename(columns = header_dict)
df_campus.head()

,CAMPUS,"Campus 2016 Index 1: Summed Grades 3-11, All Students Reading/ELA Performance Denominator","Campus 2016 Index 1: Index 1 (Phase1 Level2 & PM_ELL), Summed Grades 3-11, All Students Reading/ELA Numerator","Campus 2016 Index 1: Summed Grades 3-11, African American Reading/ELA Performance Denominator","Campus 2016 Index 1: Index 1 (Phase1 Level2 & PM_ELL), Summed Grades 3-11, African American Reading/ELA Numerator","Campus 2016 Index 1: Summed Grades 3-11, White Reading/ELA Performance Denominator","Campus 2016 Index 1: Index 1 (Phase1 Level2 & PM_ELL), Summed Grades 3-11, White Reading/ELA Numerator","Campus 2016 Index 1: Summed Grades 3-11, Hispanic Reading/ELA Performance Denominator","Campus 2016 Index 1: Index 1 (Phase1 Level2 & PM_ELL), Summed Grades 3-11, Hispanic Reading/ELA Numerator","Campus 2016 Index 1: Summed Grades 3-11, American Indian Reading/ELA Performance Denominator",...,"Index 3 Campus 2017 Performance, Masters Grade Level: Grades 3-11, Summed Econ Disadv All Subjects Rate","Index 3 Campus 2017 Performance: Grades 3-11, Summed Special Ed All Subjects Denominator","Index 3 Campus 2017 Performance, Masters Grade Level: Grades 3-11, Summed Special Ed All Subjects Numerator","Index 3 Campus 2017 Performance, Masters Grade Level: Grades 3-11, Summed Special Ed All Subjects Rate","Index 3 Campus 2017 Performance: Grades 3-11, Summed At Risk All Subjects Denominator","Index 3 Campus 2017 Performance, Masters Grade Level: Grades 3-11, Summed At Risk All Subjects Numerator","Index 3 Campus 2017 Performance, Masters Grade Level: Grades 3-11, Summed At Risk All Subjects Rate","Index 3 Campus 2017 Performance: Grades 3-11, Summed ELL All Subjects Denominator","Index 3 Campus 2017 Performance, Masters Grade Level: Grades 3-11, Summed ELL All Subjects Numerator","Index 3 Campus 2017 Performance, Masters Grade Level: Grades 3-11, Summed ELL All Subjects Rate"
0,1902001,108,78,NaN,NaN,88,70,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
1,1902041,116,100,NaN,NaN,92,80,12,9,NaN,...,10,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
2,1902103,115,104,NaN,NaN,92,88,13,9,NaN,...,18,38,7,18,NaN,NaN,NaN,NaN,NaN,NaN
3,1903001,207,173,10,6,173,147,15,13,NaN,...,6,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN
4,1903041,284,224,20,9,226,184,22,16,NaN,...,13,101,7,7,286,7,2,0,NaN,NaN


In [7]:
# use melt function to transform dataset from wide table to long table so that we can filter on the fields
df_campus = df_campus.melt(id_vars='CAMPUS', var_name='Category', value_name='Value')
print(df_campus.shape)
df_campus.head()

(13240584, 3)


,CAMPUS,Category,Value
0,1902001,"Campus 2016 Index 1: Summed Grades 3-11, All S...",108
1,1902041,"Campus 2016 Index 1: Summed Grades 3-11, All S...",116
2,1902103,"Campus 2016 Index 1: Summed Grades 3-11, All S...",115
3,1903001,"Campus 2016 Index 1: Summed Grades 3-11, All S...",207
4,1903041,"Campus 2016 Index 1: Summed Grades 3-11, All S...",284


In [8]:
# create a function 'contain' to do partial string search (case insensitive) which is used in filtering data fields
def contain(string,target):
    return string.lower() in target.lower()

In [9]:
# create a subset of the data for 2016 records only
df_campus_2016 = df_campus[df_campus['Category'].apply(lambda x: contain('2016',x))]

In [10]:
# get the total student count by campus and remove NaN
df_student_count = df_campus_2016[df_campus_2016['Category'].apply(
    lambda x: contain('All Students All Tests Performance Denominator',x))].dropna()
print(df_student_count.shape)
df_student_count.head()

(7990, 3)


,CAMPUS,Category,Value
1225980,1902001,"Campus 2016 Index 1: Summed Grades 3-11, All S...",256
1225981,1902041,"Campus 2016 Index 1: Summed Grades 3-11, All S...",337
1225982,1902103,"Campus 2016 Index 1: Summed Grades 3-11, All S...",303
1225983,1903001,"Campus 2016 Index 1: Summed Grades 3-11, All S...",484
1225984,1903041,"Campus 2016 Index 1: Summed Grades 3-11, All S...",854


In [11]:
# get the student count for those passing the exam by campus and remove NaN
df_pass_count = df_campus_2016[df_campus_2016['Category'].apply(
    lambda x: contain('All Students All Tests Numerator',x))].dropna()
print(df_pass_count.shape)
df_pass_count.head()

(7990, 3)


,CAMPUS,Category,Value
1234737,1902001,Campus 2016 Index 1: Index 1 (Phase1 Level2 & ...,201
1234738,1902041,Campus 2016 Index 1: Index 1 (Phase1 Level2 & ...,276
1234739,1902103,Campus 2016 Index 1: Index 1 (Phase1 Level2 & ...,264
1234740,1903001,Campus 2016 Index 1: Index 1 (Phase1 Level2 & ...,428
1234741,1903041,Campus 2016 Index 1: Index 1 (Phase1 Level2 & ...,657


In [12]:
# get the student pass rate by campus and remove NaN
df_pass_rate = df_campus_2016[df_campus_2016['Category'].apply(
    lambda x: contain('All Students All Tests Rate',x))].dropna()
print(df_pass_rate.shape)
df_pass_rate.head()

(7761, 3)


,CAMPUS,Category,Value
1471176,1902001,Campus 2016 Index 1: Index 1 (Phase1 Level2 & ...,79
1471177,1902041,Campus 2016 Index 1: Index 1 (Phase1 Level2 & ...,82
1471178,1902103,Campus 2016 Index 1: Index 1 (Phase1 Level2 & ...,87
1471179,1903001,Campus 2016 Index 1: Index 1 (Phase1 Level2 & ...,88
1471180,1903041,Campus 2016 Index 1: Index 1 (Phase1 Level2 & ...,77


In [13]:
# merge dataframes to include all data elements
df_result_campus = df_student_count.merge(df_pass_count,on='CAMPUS',how='outer').merge(df_pass_rate,on='CAMPUS',how='outer')
print(df_result_campus.shape)
df_result_campus.head()

(7990, 7)


,CAMPUS,Category_x,Value_x,Category_y,Value_y,Category,Value
0,1902001,"Campus 2016 Index 1: Summed Grades 3-11, All S...",256,Campus 2016 Index 1: Index 1 (Phase1 Level2 & ...,201,Campus 2016 Index 1: Index 1 (Phase1 Level2 & ...,79
1,1902041,"Campus 2016 Index 1: Summed Grades 3-11, All S...",337,Campus 2016 Index 1: Index 1 (Phase1 Level2 & ...,276,Campus 2016 Index 1: Index 1 (Phase1 Level2 & ...,82
2,1902103,"Campus 2016 Index 1: Summed Grades 3-11, All S...",303,Campus 2016 Index 1: Index 1 (Phase1 Level2 & ...,264,Campus 2016 Index 1: Index 1 (Phase1 Level2 & ...,87
3,1903001,"Campus 2016 Index 1: Summed Grades 3-11, All S...",484,Campus 2016 Index 1: Index 1 (Phase1 Level2 & ...,428,Campus 2016 Index 1: Index 1 (Phase1 Level2 & ...,88
4,1903041,"Campus 2016 Index 1: Summed Grades 3-11, All S...",854,Campus 2016 Index 1: Index 1 (Phase1 Level2 & ...,657,Campus 2016 Index 1: Index 1 (Phase1 Level2 & ...,77


In [14]:
# select columns and rename column headers, add a column for school year 2016
df_result_campus = df_result_campus[['CAMPUS','Value_x','Value_y','Value']].rename(
    columns = {'CAMPUS':'Campus','Value_x':'Student_count','Value_y':'Pass_count','Value':'Pass_rate'})
df_result_campus['Year'] = 2016
print(df_result_campus.shape)
df_result_campus.head()

(7990, 5)


,Campus,Student_count,Pass_count,Pass_rate,Year
0,1902001,256,201,79,2016
1,1902041,337,276,82,2016
2,1902103,303,264,87,2016
3,1903001,484,428,88,2016
4,1903041,854,657,77,2016


In [15]:
# note there are 7990 schools but only 7761 schools have reported with pass rate.
# the delta represents the schools with small number of students and pass rate data are masked by TEA
# if we decide to include all schools, we can calucluate the pass rate by ourselves, but I am afraid it may distort the data
# for example, 50% pass rate for a school with 2 students may look high, but actually not very representative
print(df_result_campus['Campus'].count())
print(df_result_campus['Pass_rate'].count())

7990
7761


In [16]:
# remove schools without pass rate
df_result_campus = df_result_campus.dropna()
df_result_campus.shape

(7761, 5)

In [17]:
# save data to csv file
df_result_campus.to_csv('result_campus.csv',index=False)